<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/instruct_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

In [1]:
# 필수 라이브러리 임포트
import json            # JSON 데이터 구문 분석을 위한 라이브러리
import random          # 시드 설정 및 데이터 셔플링을 위한 라이브러리
import requests        # URL에서 데이터셋 다운로드를 위한 라이브러리
import torch           # 메인 파이토치 라이브러리
from torch.utils.data import Dataset, DataLoader  # 데이터셋 처리를 위한 라이브러리
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria  # 허깅 페이스 구성 요소
from tqdm import tqdm   # 진행률 표시줄 유틸리티
import re               # 텍스트 정규화를 위한 라이브러리

def set_seed(seed):
    """
    다양한 라이브러리에서 재현성을 위해 난수 시드를 설정합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값
    """
    # 파이썬 내장 random 시드 설정
    random.seed(seed)
    # 파이토치 CPU 난수 시드 설정
    torch.manual_seed(seed)
    # 사용 가능한 모든 GPU에 대한 시드 설정
    torch.cuda.manual_seed_all(seed)
    # cuDNN이 결정론적 알고리즘을 사용하도록 요청
    torch.backends.cudnn.deterministic = True
    # 일관된 동작을 위해 cuDNN의 자동 튜너 비활성화
    torch.backends.cudnn.benchmark = False

def build_prompt(instruction, solution=None):
    """
    시스템, 사용자 및 어시스턴트 메시지가 있는 채팅 형식의 프롬프트를 생성합니다.
    Args:
        instruction (str): 사용자의 지시/질문
        solution (str, optional): 훈련을 위한 예상 응답
    Returns:
        str: 형식화된 프롬프트 문자열
    """
    # 제공된 경우 종료 토큰과 함께 솔루션 추가
    wrapped_solution = ""
    if solution:
        wrapped_solution = f"\n{solution}\n<|im_end|>"
    # 시스템, 사용자 및 어시스턴트 메시지로 채팅 형식 구축
    return f"""<|im_start|>system
You are a helpful assistant.
<|im_end|>
<|im_start|>user
{instruction}
<|im_end|>
<|im_start|>assistant""" + wrapped_solution

def encode_text(tokenizer, text, return_tensor=False):
    """
    제공된 토크나이저를 사용하여 텍스트를 인코딩합니다.
    Args:
        tokenizer: 허깅 페이스 토크나이저
        text (str): 인코딩할 텍스트
        return_tensor (bool): 파이토치 텐서를 반환할지 여부
    Returns:
        토큰 ID의 목록 또는 텐서
    """
    # 텐서 출력이 요청된 경우, 파이토치 텐서로 인코딩
    if return_tensor:
        return tokenizer.encode(
            text, add_special_tokens=False, return_tensors="pt"
        )
    # 그렇지 않으면 토큰 ID 목록 반환
    else:
        return tokenizer.encode(text, add_special_tokens=False)

class EndTokenStoppingCriteria(StoppingCriteria):
    """
    텍스트 생성을 위한 사용자 정의 중단 기준.
    특정 종료 토큰 시퀀스가 생성되면 중지합니다.
    Args:
        end_tokens (list): 생성이 중지되어야 함을 나타내는 토큰 ID
        device: 모델이 실행 중인 장치
    """
    def __init__(self, end_tokens, device):
        self.end_tokens = torch.tensor(end_tokens).to(device)

    def __call__(self, input_ids, scores):
        """
        각 시퀀스에 대해 생성을 중지해야 하는지 확인합니다.
        Args:
            input_ids: 현재 생성된 토큰 ID
            scores: 토큰 확률
        Returns:
            tensor: 어떤 시퀀스를 중지해야 하는지 나타내는 부울 텐서
        """
        should_stop = []
        # 종료 토큰에 대해 각 시퀀스 확인
        for sequence in input_ids:
            if len(sequence) >= len(self.end_tokens):
                # 마지막 토큰을 종료 토큰과 비교
                last_tokens = sequence[-len(self.end_tokens):]
                should_stop.append(torch.all(last_tokens == self.end_tokens))
            else:
                should_stop.append(False)
        return torch.tensor(should_stop, device=input_ids.device)

class PromptCompletionDataset(Dataset):
    """
    지시-완성 쌍을 위한 파이토치 데이터셋.
    텍스트 데이터를 모델 준비 형식으로 변환하는 작업을 처리합니다.
    Args:
        data (list): 지시와 솔루션을 포함하는 사전 목록
        tokenizer: 허깅 페이스 토크나이저
    """
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        # 총 샘플 수 반환
        return len(self.data)

    def __getitem__(self, idx):
        """
        단일 훈련 샘플을 반환합니다.
        Args:
            idx (int): 가져올 샘플의 인덱스
        Returns:
            dict: input_ids, labels, prompt, expected_completion을 포함
        """
        # 데이터셋에서 샘플 가져오기
        item = self.data[idx]
        # 지시와 함께 전체 프롬프트 구축
        prompt = build_prompt(item["instruction"])
        # 종료 토큰으로 완성 형식 지정
        completion = f"""{item["solution"]}\n<|im_end|>"""
        # 텍스트를 토큰 ID로 변환
        encoded_prompt = encode_text(self.tokenizer, prompt)
        encoded_completion = encode_text(self.tokenizer, completion)
        eos_token = [self.tokenizer.eos_token_id]
        # 전체 입력 시퀀스를 위해 결합
        input_ids = encoded_prompt + encoded_completion + eos_token
        # 레이블 생성: 프롬프트의 경우 -100(손실에서 무시됨)
        labels = [-100] * len(encoded_prompt) + encoded_completion + eos_token
        return {
            "input_ids": input_ids,
            "labels": labels,
            "prompt": prompt,
            "expected_completion": completion
        }

def collate_fn(batch):
    """
    샘플 배치를 훈련 준비 형식으로 통합합니다.
    패딩과 텐서 변환을 처리합니다.
    Args:
        batch: 데이터셋의 샘플 목록
    Returns:
        tuple: (input_ids, attention_mask, labels, prompts, expected_completions)
    """
    # 패딩을 위해 가장 긴 시퀀스 찾기
    max_length = max(len(item["input_ids"]) for item in batch)
    # 입력 시퀀스 패딩
    input_ids = [
        item["input_ids"] +
        [tokenizer.pad_token_id] * (max_length - len(item["input_ids"]))
        for item in batch
    ]
    # 레이블 시퀀스 패딩
    labels = [
        item["labels"] +
        [-100] * (max_length - len(item["labels"]))
        for item in batch
    ]
    # 어텐션 마스크 생성
    attention_mask = [
        [1] * len(item["input_ids"]) +
        [0] * (max_length - len(item["input_ids"]))
        for item in batch
    ]
    prompts = [item["prompt"] for item in batch]
    expected_completions = [item["expected_completion"] for item in batch]
    return (
        torch.tensor(input_ids),
        torch.tensor(attention_mask),
        torch.tensor(labels),
        prompts,
        expected_completions
    )

def normalize_text(text):
    """
    일관된 비교를 위해 텍스트를 정규화합니다.
    Args:
        text (str): 입력 텍스트
    Returns:
        str: 정규화된 텍스트
    """
    # 선행/후행 공백 제거 및 소문자로 변환
    text = text.strip().lower()
    # 여러 공백 문자를 단일 공백으로 대체
    text = re.sub(r'\s+', ' ', text)
    return text

def generate_text(model, tokenizer, prompt, max_new_tokens=100):
    """
    주어진 프롬프트에 대한 텍스트 완성을 생성합니다.
    Args:
        model: 미세 튜닝된 모델
        tokenizer: 연관된 토크나이저
        prompt (str): 입력 프롬프트
        max_new_tokens (int): 생성할 최대 토큰 수
    Returns:
        str: 생성된 완성
    """
    # 프롬프트 인코딩 및 모델 장치로 이동
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    # 종료 토큰 감지 설정
    end_tokens = tokenizer.encode("<|im_end|>", add_special_tokens=False)
    stopping_criteria = [EndTokenStoppingCriteria(end_tokens, model.device)]
    # 완성 생성
    output_ids = model.generate(
        input_ids=input_ids["input_ids"],
        attention_mask=input_ids["attention_mask"],
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        stopping_criteria=stopping_criteria
    )[0]
    # 생성된 부분만 추출 및 디코딩
    generated_ids = output_ids[input_ids["input_ids"].shape[1]:]
    generated_text = tokenizer.decode(generated_ids).strip()
    return generated_text

def test_model(model_path, test_input):
    """
    저장된 모델을 단일 입력으로 테스트합니다.
    Args:
        model_path (str): 저장된 모델의 경로
        test_input (str): 테스트할 지시
    """
    # 장치 설정 및 모델 로드
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 장치: {device}")
    # 모델과 토크나이저 로드
    model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    # 예측 생성 및 표시
    prompt = build_prompt(test_input)
    generated_text = generate_text(model, tokenizer, prompt)
    print(f"\n입력: {test_input}")
    print(f"전체 생성된 텍스트: {generated_text}")
    print(f"""응답: {generated_text.replace("<|im_end|>", "").strip()}""")

def download_and_prepare_data(data_url, tokenizer, batch_size, test_ratio=0.1):
    """
    훈련을 위해 데이터셋을 다운로드하고 준비합니다.
    Args:
        data_url (str): 데이터셋의 URL
        tokenizer: 텍스트 처리를 위한 토크나이저
        batch_size (int): DataLoader의 배치 크기
        test_ratio (float): 테스트를 위한 데이터 비율
    Returns:
        tuple: (train_loader, test_loader)
    """
    # 데이터셋 다운로드
    response = requests.get(data_url)
    dataset = []
    # 각 라인을 지시-솔루션 쌍으로 구문 분석
    for line in response.text.splitlines():
        if line.strip():  # 빈 라인 건너뛰기
            entry = json.loads(line)
            dataset.append({
                "instruction": entry["instruction"],
                "solution": entry["solution"]
            })
    # 훈련 및 테스트 세트로 분할
    random.shuffle(dataset)
    split_index = int(len(dataset) * (1 - test_ratio))
    train_data = dataset[:split_index]
    test_data = dataset[split_index:]
    # 데이터셋 통계 출력
    print(f"\n데이터셋 크기: {len(dataset)}")
    print(f"훈련 샘플: {len(train_data)}")
    print(f"테스트 샘플: {len(test_data)}")
    # 데이터셋 생성
    train_dataset = PromptCompletionDataset(train_data, tokenizer)
    test_dataset = PromptCompletionDataset(test_data, tokenizer)
    # 데이터로더 생성
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )
    return train_loader, test_loader

def get_hyperparameters():
    """
    훈련 하이퍼파라미터를 반환합니다.
    Returns:
        tuple: (num_epochs, batch_size, learning_rate)
    """
    # 지시 미세 튜닝은 더 데이터 효율적이므로 더 적은 에포크
    num_epochs = 4
    # 대부분의 GPU 메모리에서 잘 작동하는 표준 배치 크기
    batch_size = 16
    # 트랜스포머 미세 튜닝을 위한 표준 학습률
    learning_rate = 5e-5
    return num_epochs, batch_size, learning_rate

In [2]:
# 재현성을 위해 난수 시드 설정
set_seed(42)

# 훈련 매개변수 구성
data_url = "https://www.thelmbook.com/data/instruct"
model_name = "openai-community/gpt2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

# 토크나이저와 모델 초기화
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# 하이퍼파라미터 가져오기 및 데이터 준비
num_epochs, batch_size, learning_rate = get_hyperparameters()
train_loader, test_loader = download_and_prepare_data(data_url, tokenizer, batch_size)

# 옵티마이저 초기화
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 훈련 루프
for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 0
    progress_bar = tqdm(train_loader, desc=f"에포크 {epoch+1}/{num_epochs}")

    for input_ids, attention_mask, labels, _, _ in progress_bar:
        # 배치를 장치로 이동
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 포워드 패스
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

        # 백워드 패스 및 최적화
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 메트릭 업데이트
        total_loss += loss.item()
        num_batches += 1
        progress_bar.set_postfix({"손실": total_loss / num_batches})

    # 에포크 메트릭 표시
    avg_loss = total_loss / num_batches
    print(f"에포크 {epoch+1} - 평균 손실: {avg_loss:.4f}")

# 미세 튜닝된 모델 저장
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

# 모델 테스트
print("\n미세 튜닝된 모델 테스트:")
test_input = "Who is the President of the United States?"
test_model("./finetuned_model", test_input)

사용 중인 장치: cuda


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

에포크 2 - 평균 손실: 1.0221


에포크 3/4: 100%|██████████| 29/29 [00:03<00:00,  7.92it/s, 손실=0.688]


에포크 3 - 평균 손실: 0.6885


에포크 4/4: 100%|██████████| 29/29 [00:03<00:00,  7.79it/s, 손실=0.422]


에포크 4 - 평균 손실: 0.4221

미세 튜닝된 모델 테스트:
사용 중인 장치: cuda

입력: Who is the President of the United States?
전체 생성된 텍스트: George W. Bush
<|im_end|>
응답: George W. Bush
